<a href="https://colab.research.google.com/github/Abhiyantrana-Navonmesakah/Wealthwise_backend/blob/main/Multi_agent_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install serpapi dotenv crewai pydantic transformers torch
# !pip install crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 927.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s

In [ ]:
import os
os.environ["GROQ_API_KEY"]="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"

In [ ]:
from typing import Type, Literal
from typing import Dict, List, Union
from pydantic import BaseModel, Field
from crewai.tools import BaseTool
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import re
import yfinance as yf
import json
class StockReportInput(BaseModel):
    """Input schema for Stock Report Tool."""
    query_name: Literal["Momentum-Trading", "Scalping", "Position-trading", "swing-trading", "Day-trading"] = Field(
        ...,
        description="Trading strategy for stock screening. Must be one of: 'Momentum-Trading', 'Scalping', 'Position-trading', 'swing-trading', 'Day-trading'"
    )

class StockReportTool(BaseTool):
    """Tool for generating stock reports based on different trading strategies."""

    name: str = "Stock Report Tool"
    description: str = """
    Generates a stock report based on specific trading strategies.
    Available strategies: 'Momentum-Trading', 'Scalping', 'Position-trading', 'swing-trading', 'Day-trading'
    Returns a detailed report with stock metrics and financial data for the top matches.
    """
    args_schema: Type[BaseModel] = StockReportInput

    def _get_symbol_from_bse(self, company_id):
        """Fetch symbol from BSE API."""
        BSE_API_URL = "https://api.bseindia.com/Msource/1D/getQouteSearch.aspx?Type=EQ&text={}&flag=site"
        url = BSE_API_URL.format(company_id)
        headers = {
            "Referer": "https://www.bseindia.com/",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            match = re.search(r"<span>\s*([\w-]+)\s*&nbsp;", response.text)
            if match:
                return match.group(1)  # Extract the stock symbol
        return "Unknown"

    def _get_stock_metrics(self, ticker_symbol):
        """Fetch stock metrics from Yahoo Finance."""
        try:
            stock = yf.Ticker(ticker_symbol)
            info = stock.info
            history = stock.history(period="5y")  # Get 5 years of historical data

            def calculate_growth(current, past):
                if current and past and past != 0:
                    return ((current - past) / past) * 100
                return "N/A"

            try:
                return_1y = calculate_growth(history['Close'].iloc[-1], history['Close'].iloc[-252]) if len(history) > 252 else "N/A"
                return_3y = calculate_growth(history['Close'].iloc[-1], history['Close'].iloc[-756]) if len(history) > 756 else "N/A"
            except (KeyError, IndexError):
                return_1y = return_3y = "N/A"

            metrics = {
                "Return over 1 Year": return_1y,
                "Return over 3 Years": return_3y,
                "P/B Ratio": info.get("priceToBook", "N/A"),
                "EV/EBITDA": info.get("enterpriseToEbitda", "N/A"),
                "Dividend Yield": info.get("dividendYield", "N/A"),
                "P/S Ratio": info.get("priceToSalesTrailing12Months", "N/A"),
                "ROE": info.get("returnOnEquity", "N/A"),
                "Market Cap": info.get("marketCap", "N/A"),
                "Debt/Equity Ratio": info.get("debtToEquity", "N/A")
            }
            return metrics

        except Exception as e:
            print(f"Error fetching data for {ticker_symbol}: {e}")
            return None

    def _run(self, query_name: str) -> str:
        """
        Generate a stock report based on the specified trading strategy.

        Args:
            query_name: The trading strategy to use. Must be one of:
                        'Momentum-Trading', 'Scalping', 'Position-trading', 'swing-trading', 'Day-trading'

        Returns:
            A formatted stock report as a string
        """
        # Print debug info
        print(f"Executing StockReportTool with query_name: {query_name}")

        # Queries dictionary with screening conditions for each strategy
        queries = {
            "Day-trading": """EPS latest quarter > 1.2 * EPS preceding year quarter AND
                EPS latest quarter > 0 AND
                YOY Quarterly sales growth > 25 AND
                EPS last year > EPS preceding year AND
                EPS > EPS last year AND
                Profit growth 3Years > 25 AND
                Return on equity > 17 AND
                Down from 52w high < 18 AND
                Market Capitalization > 5 AND
                ( Current assets / Current liabilities) > 2""",
            "swing-trading": """PEG Ratio > 2 AND
                Debt to equity < 0.5 AND
                Market Capitalization > 5000 AND
                52w Index >75 AND
                Average return on equity 5Years > 0""",
            "Position-trading": """Market Capitalization  > 1000 AND
                OPM 5Year >Price to Earning AND
                Interest Coverage Ratio >3 AND Debt
                to equity < 1 AND Pledged percentage =0""",
            "Scalping": """Volume 1week average > 2*(Volume 1month average) AND Volume > 1000000""",
            "Momentum-Trading": """Down from 52w high <= 6 AND
                Current price > 20 AND
                net profit > 30 and
                Market Capitalization > 500""",
            "Long-term-investment":"""Market Capitalization >3000 AND
                Return on invested capital >15 AND
                Average return on capital employed 5Years >14 AND
                Average return on equity 5Years >14 AND
                Average return on capital employed 3Years >14 AND
                Average return on equity 3Years >14 AND
                Sales growth 5Years >8 AND
                Sales growth 3Years >8 AND
                Profit growth 5Years >10 AND
                Profit growth 3Years >14 AND
                Debt to equity <0.6 AND
                Current ratio >1"""
        }

        query = queries.get(query_name)
        if query is None:
            return f"Invalid query name: {query_name}. Please use one of: 'Momentum-Trading', 'Scalping', 'Position-trading', 'swing-trading', 'Day-trading'"

        # Base URL for the screener
        base_url = "https://www.screener.in/screen/raw/"

        # Query parameters
        payload = {"sort": "", "order": "", "source_id": "107002", "query": query}

        # Headers (Only 'Cookie' included)
        headers = {"Cookie": "csrftoken=XsBj6oYoyw12I5hBbi7khW2rYknPw549; sessionid=ursq4btr8kr1pp9ssdnxzji5fshwdbsw"}  # Replace with your actual cookie

        # Sending GET request
        response = requests.get(base_url, params=payload, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table')

            if table:
                rows = table.find_all('tr')
                headers = ["Symbol"] + [th.text.strip() for th in rows[0].find_all('th')]
                data = []

                for row in rows[1:min(len(rows), 11)]:
                    cells = row.find_all('td')
                    row_data = [cell.text.strip() for cell in cells]

                    symbol = "-"
                    link = row.find('a', href=True)
                    if link:
                        href = link['href']
                        match1 = re.search(r"/company/([^/]+)/consolidated/", href)
                        match2 = re.search(r"/company/([^/]+)/?$", href)
                        match3 = re.search(r"/company/(\d+)", href)

                        if match1:
                            symbol = match1.group(1) + ".NS"
                        elif match3:
                            company_id = match3.group(1)
                            symbol = self._get_symbol_from_bse(company_id) + ".BO"
                        elif match2:
                            symbol = match2.group(1) + ".NS"

                    stock_metrics = (self._get_stock_metrics(symbol) or {}) if symbol != "Unknown" else {}
                    metrics_values = [stock_metrics.get(metric, "N/A") for metric in
                                    ["Return over 1 Year", "Return over 3 Years", "P/B Ratio", "EV/EBITDA",
                                    "Dividend Yield", "P/S Ratio", "ROE", "Market Cap", "Debt/Equity Ratio"]]
                    data.append([symbol] + row_data + metrics_values)
                    # data.append([symbol] + row_data)

                headers += ["Return over 1 Year", "Return over 3 Years", "P/B Ratio", "EV/EBITDA", "Dividend Yield",
                            "P/S Ratio", "ROE", "Market Cap", "Debt/Equity Ratio"]
                report1 = f"""
                ****** STOCK REPORT: {query_name} ******

                Query Conditions:
                -----------------
                {query}

                Retrieved {len(data)} stock entries.

                Data Table:
                -----------
                {tabulate(data, headers=headers, tablefmt="grid")}

                *****************
                """
                print(report1)
                data=data[:7]

                report_data = {
                    "query_name": query_name,
                    "query": queries.get(query_name),
                    "retrieved_count": len(data),
                    "data": data
                }
                report_json = json.dumps(report_data, indent=None, default=str)  # indent=None for minimal size

                return report_json
            else:
                return "No table found on the page."
        else:
            return f"Failed to retrieve data. Status code: {response.status_code}"

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from serpapi import Client
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)
serp_client = Client(api_key="c95afc4816ed3d891485699ed294ee7309f763d2a5790c87ad4891330b2b6a0a")

Device set to use cpu


In [ ]:
def print_sentiment_report(sentiment_reports):
    """Prints sentiment reports in a formatted, readable way."""
    for company, data in sentiment_reports.items():
        print("-" * 50)  # Separator between companies
        print(f"Company: {data['company_name']}")
        print("-" * 50)

        print(f"Status: {data['status']}")
        print(f"Reputation Score: {data['reputation_score']}")

        print("\nSentiment Distribution:")
        for sentiment, percentage in data['sentiment_distribution'].items():
            print(f"  {sentiment}: {percentage:.2f}%")  # Format percentage

        print("\nTop Articles:")
        for i, article in enumerate(data['top_articles']):
            print(f"  {i+1}. Title: {article['title']}")
            if article.get('link'):  # Check if link exists
                print(f"     Link: {article['link']}")
            print(f"     Sentiment: {article['sentiment']}")
            print(f"     Score: {article['score']:.4f}")  # Format score
            print("-" * 20)  # Separator between articles
        print("\n")  # Space between company reports

class SentimentAnalysisInput(BaseModel):
    """Input schema for Sentiment Analysis Tool."""
    companies: List[str] = Field(
        ...,
        description="List of company names to analyze sentiment for."
    )
class CompanySentimentTool(BaseTool):
    """Tool for analyzing market sentiment and company reputation from news articles."""

    name: str = "Company Sentiment Analysis Tool"
    description: str = """
    Analyzes market sentiment and reputation for companies based on recent news articles.
    Uses FinBERT model for sentiment analysis and retrieves news from Google News.
    Returns sentiment scores, reputation analysis, and key articles for each company.
    """
    args_schema: Type[BaseModel] = SentimentAnalysisInput

    def _run(self, companies: List[str]) -> dict:
        """
        Run sentiment analysis for the specified companies.

        Args:
            companies: List of company names to analyze

        Returns:
            Dictionary containing sentiment analysis results for each company
        """
        sentiment_reports = {}
        sentiment_reports1 = {}
        num_articles = 10

        for company in companies:
            try:
                # Fetch news articles
                results = serp_client.search({
                    'engine': 'google_news',
                    'q': company,
                    'gl': 'in',
                    'hl': 'en'
                })

                news_articles = results.get('news_results', [])[:num_articles]

                if not news_articles:
                    sentiment_reports[company] = {
                        "status": "error",
                        "message": f"No news articles found for {company}"
                    }
                    continue

                # Analyze sentiment
                sentiment_counts = {'Positive': 0, 'Neutral': 0, 'Negative': 0}
                analyzed_articles = []

                for article in news_articles:
                    title = article['title']
                    sentiment_result = nlp(title)[0]
                    sentiment = sentiment_result['label']
                    score = sentiment_result['score']

                    sentiment_counts[sentiment] += 1
                    analyzed_articles.append({
                        'title': title,
                        'link': article.get('link', ''),
                        'sentiment': sentiment,
                        'score': round(score, 4)
                    })

                # Calculate metrics
                total_articles = len(news_articles)
                sentiment_distribution = {
                    sentiment: round((count / total_articles) * 100, 2)
                    for sentiment, count in sentiment_counts.items()
                }

                # Calculate reputation score
                sentiment_weights = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
                reputation_score = sum([sentiment_weights[art['sentiment']] for art in analyzed_articles])
                reputation_score = (reputation_score / total_articles) * 50 + 50

                # Store results
                sentiment_reports[company] = {
                    "company_name": company,
                    "reputation_score": round(reputation_score, 2),
                    "sentiment_distribution": sentiment_distribution,
                    "top_articles": [article['title'] for article in analyzed_articles[:3]]
                }
                sentiment_reports1[company] = {
                    "status": "success",
                    "company_name": company,
                    "reputation_score": round(reputation_score, 2),
                    "sentiment_distribution": sentiment_distribution,
                    "top_articles": analyzed_articles[:3]
                }

                # Print the sentiment report for this company
                print_sentiment_report({company: sentiment_reports1[company]})

            except Exception as e:
                sentiment_reports[company] = {
                    "status": "error",
                    "message": f"Error analyzing {company}: {str(e)}"
                }

        # Print the final sentiment reports
        print("Final Sentiment Reports:")
        print(json.dumps(sentiment_reports1, indent=2))
        return sentiment_reports

In [ ]:
from crewai import Agent, Task, Crew, LLM, Process

# Define LLMs
llm_70b = LLM(model="groq/llama-3.3-70b-versatile")  # For Financial Metrics Agent
llm_8b = LLM(model="groq/llama-3.1-8b-instant")  # For Financial Analyst Agent and Crew Manager
llm_32b = LLM(model="groq/deepseek-r1-distill-qwen-32b")  # For Sentiment and Recommendation Agent

# Define Tools
stock_report_tool = StockReportTool()
sentiment_tool = CompanySentimentTool()

# Define Agents
financial_analyst_agent = Agent(
    role="Financial Analyst",
    goal="Analyze the user's financial condition and determine the budget and risk level for investment.",
    backstory="""You are a seasoned financial analyst with expertise in evaluating financial conditions and
    determining appropriate investment strategies based on income, expenses, savings, and risk tolerance.""",
    llm=llm_8b,
    verbose=True
)

financial_metrics_agent = Agent(
    role="Financial Metrics Analyst",
    goal="Fetch a list of stocks with financial metrics based on the user's strategy, risk tolerance, and financial analysis.",
    backstory="""You are a financial metrics expert who specializes in analyzing stocks based on financial data.
    You use advanced tools to fetch and evaluate stock metrics, ensuring the recommendations align with the user's strategy and risk tolerance.""",
    tools=[stock_report_tool],
    llm=llm_70b,
    verbose=True
)

sentiment_recommendation_agent = Agent(
    role="Sentiment and Recommendation Analyst",
    goal="Analyze market sentiment and recommend the top 5 stocks based on financial metrics and sentiment analysis.",
    backstory="""You are an expert in analyzing market sentiment and company reputation. You combine financial metrics
    and sentiment analysis to provide well-rounded stock recommendations that align with the user's strategy and risk tolerance.""",
    tools=[sentiment_tool],
    llm=llm_32b,
    verbose=True
)

# Define Tasks
financial_analysis_task = Task(
    description="""Analyze the user's financial condition metrics and determine the budget and risk level for investment.
    User financial metrics:
    - Income: {income}
    - Expenses: {expenses}
    - Savings: {savings}
    - Investment Amount: {investment_amount}
    - Risk Tolerance: {risk_tolerance}

    Provide a concise analysis of the user's financial health and recommend a budget and risk level for investment.
    Include 2-3 key conclusions.
    """,
    agent=financial_analyst_agent,
    expected_output="""A concise report containing:
    1. Input details (income, expenses, savings, investment amount, risk tolerance).
    2. 2-3 key conclusions (e.g., recommended budget, risk level).
    """
)

financial_metrics_task = Task(
    description="""Based on the previous Financial Analyst's recommendations, fetch and analyze appropriate stocks.

    Previous Analysis: {{financial_analysis_task.output}}

    User Strategy: {strategy}
    User Risk Tolerance: {risk_tolerance}

    Use the StockReportTool to generate a report with financial metrics for the top stocks.
    Provide detailed insights about why these stocks were selected, including:
    - Key financial metrics (e.g., P/E ratio, dividend yield, market cap)
    - Alignment with the user's strategy and risk tolerance
    - Growth potential and recent performance

    **Important:** Ensure the output includes both the stock tickers and their corresponding names.
    """,
    agent=financial_metrics_agent,
    expected_output="""A detailed report containing:
    1. Names of the top stocks.
    2. Stock tickers.
    3. Key financial metrics for each stock.
    4. Explanation of why these stocks were selected.
    """
)

sentiment_recommendation_task = Task(
    description="""Analyze market sentiment and recommend the top 5 stocks based on financial metrics and sentiment analysis.

    Inputs:
    - List of stock names and insights: {{financial_metrics_task.output}}

    **Important:** Ensure the CompanySentimentTool is used to analyze sentiment and reputation for all stocks at once.
    Log the tool's input and output for verification.
    Rank the stocks based on both financial metrics and sentiment scores.
    For each stock, provide a detailed explanation, including:
    1. **Summary of Financial Metrics:**
       - P/E Ratio, Dividend Yield, Market Capitalization, and other relevant metrics.
    2. **Summary of Sentiment Analysis:**
       - Sentiment score (if available).
       - Key news highlights and their sentiment (positive, neutral, negative).
    3. **Reasoning for Selection:**
       - Link financial metrics and sentiment analysis to the user's strategy and risk tolerance.
    4. **Risk Assessment:**
       - Evaluate the risk level (low, medium, high) and alignment with the user's risk tolerance.

    **Note:** If sentiment data is unavailable for a stock, provide a detailed explanation based on financial metrics alone.
    Ensure the output is structured and easy to understand.
    """,
    agent=sentiment_recommendation_agent,
    expected_output="""A detailed report containing:
    1. Top 5 recommended stocks.
    2. For each stock:
       - Summary of financial metrics.
       - Summary of sentiment analysis (if available).
       - Detailed reasoning for selection.
       - Risk assessment and alignment with the user's strategy.
    3. A conclusion summarizing the recommendations and their alignment with the user's goals.
    """
)

# Create crew to manage agents and task workflow
financial_crew = Crew(
    agents=[financial_analyst_agent, financial_metrics_agent, sentiment_recommendation_agent],  # Include all agents
    tasks=[financial_analysis_task, financial_metrics_task, sentiment_recommendation_task],  # Tasks in execution order
    verbose=True,
    process=Process.sequential,
    manager_llm=llm_8b  # Use the smaller model for crew management
)

# Define the User's Financial Condition Metrics
user_financial_metrics = {
    "income": 250000,
    "expenses": 50000,
    "savings": 200000,
    "investment_amount": 100000,
    "risk_tolerance": "medium",
    "strategy": "Swing-trading"
}


financial_analysis_report = financial_crew.kickoff(inputs=user_financial_metrics)

# Agent: Financial Analyst
## Task: Analyze the user's financial condition metrics and determine the budget and risk level for investment.
    User financial metrics:
    - Income: 250000
    - Expenses: 50000
    - Savings: 200000
    - Investment Amount: 100000
    - Risk Tolerance: medium

    Provide a concise analysis of the user's financial health and recommend a budget and risk level for investment.
    Include 2-3 key conclusions.
    


# Agent: Financial Analyst
## Final Answer: 
**User Financial Condition Report**

**Input Details:**

- Income: $250,000 per year
- Expenses: $50,000 per year
- Savings: $200,000
- Investment Amount: $100,000
- Risk Tolerance: Medium

**Analysis and Recommendations:**

**1. Financial Health Analysis:**
The user has a relatively high income of $250,000 per year, which provides a good financial foundation. However, there is a significant disparity between their income and expenses, indicating that they have substantial disposable income. Their sa

In [ ]:
from IPython.display import Markdown

print("Final Recommendation Report:")
text_output = financial_analysis_report.raw
markdown_output = Markdown(text_output)
markdown_text = markdown_output.data
print(markdown_output.data)

Final Recommendation Report:
Top 5 Stock Recommendations Based on Sentiment and Financial Metrics

**1. ANANDRATHI.NS - Anand Rathi Wea.**
- **Sentiment Score:** 0.85 (Positive)
- **Financial Metrics:**
  - Market Capitalization: 16783.50
  - P/E Ratio: 106.78
  - Dividend Yield: 0.71
  - Return on Equity: 18.39
- **Analysis:** Anand Rathi Wea. has shown strong financial performance with a high return on equity and positive sentiment. The company's positive news highlights include strategic growth initiatives and strong quarterly results.
- **Risk Assessment:** Moderate risk with potential for high returns. Suitable for medium-risk tolerance investors.

**2. GANESHHOUC.NS - Ganesh Housing**
- **Sentiment Score:** 0.78 (Positive)
- **Financial Metrics:**
  - Market Capitalization: 10249.84
  - P/E Ratio: 58.66
  - Dividend Yield: 3.18
  - Return on Equity: 19.29
- **Analysis:** Ganesh Housing has a strong financial foundation with high return on equity and positive sentiment. Recent new

In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask, request
from pyngrok import ngrok
# from flask_cors import CORS
# CORS(app, resources={r"/stockRecommandation": {"origins": "https://wealthwise.vercel.app"}})

app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello, World!"

# POST route to print body content
@app.route('/stockRecommandation', methods=['POST'])
def stock_recommendation():
    try:
        content = request.json
        print(f"Received content: {content['input']}")
        result = financial_crew.kickoff(inputs=content['input'])
        return {"status": "success", "received": result.raw}, 200
    except Exception as e:
        print(f"Error: {e}")
        return {"status": "error", "message": str(e)}, 500

# Start ngrok tunnel
NGROK_DOMAIN = "keen-marten-tops.ngrok-free.app"
ngrok.set_auth_token("2lGTrPfKaseITcmgdp5ZXnaCrl6_2ABHvnjSSZ9WpLFvV8a2u")

tunnel_config = {
    "addr": 5000,
    "hostname": NGROK_DOMAIN
}
public_url = ngrok.connect(**tunnel_config)
print(f"Public URL: {public_url}")  # Specify port as an integer and protocol as "http"
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

if __name__ == '__main__':
    app.run(port=5000)

Public URL: NgrokTunnel: "https://keen-marten-tops.ngrok-free.app" -> "http://localhost:5000"
 * ngrok tunnel "NgrokTunnel: "https://keen-marten-tops.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received content: {'income': '50000', 'expenses': '30000', 'savings': '20000', 'investment_amount': '10000', 'risk_tolerance': 'High', 'strategy': 'swing-trading'}
# Agent: Financial Analyst
## Task: Analyze the user's financial condition metrics and determine the budget and risk level for investment.
    User financial metrics:
    - Income: 50000
    - Expenses: 30000
    - Savings: 20000
    - Investment Amount: 10000
    - Risk Tolerance: High

    Provide a concise analysis of the user's financial health and recommend a budget and risk level for investment.
    Include 2-3 key conclusions.
    


# Agent: Financial Analyst
## Final Answer: 
**Financial Analysis Report**

**User Financial Metrics:**

- **Income:** 50,000
- **Expenses:** 30,000
- **Savings:** 20,000
- **Investment Amount:** 10,000
- **Risk Tolerance:** High

**Key Conclusions:**

1. **Financial Health:** Based on the user's income and expenses, their monthly disposable income is $20,000 ($50,000 - $30,000). This ind

INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 05:00:36] "POST /stockRecommandation HTTP/1.1" 200 -




# Agent: Sentiment and Recommendation Analyst
## Final Answer: 
Based on the sentiment analysis and financial metrics, the top 5 recommended stocks are WIPRO.NS, BSE.NS, AVANTIFEED.NS, HEXT.NS, and DIVISLAB.NS. WIPRO.NS has a strong reputation and positive sentiment, while BSE.NS has moderate sentiment but stable financials. AVANTIFEED.NS and HEXT.NS show potential growth despite limited sentiment data. DIVISLAB.NS is included for its market presence, though caution is advised due to limited news coverage. These selections balance financial performance with market sentiment, aligning with the user's high-risk tolerance strategy.
```


Received content: {'income': '20000', 'expenses': '18000', 'savings': '2000', 'investment_amount': '4000', 'risk_tolerance': 'High', 'strategy': 'swing-trading'}
# Agent: Financial Analyst
## Task: Analyze the user's financial condition metrics and determine the budget and risk level for investment.
    User financial metrics:
    - Income: 20000
    - 

...

Error fetching data for 530245.NS: 'NoneType' object has no attribute 'update'

                ****** STOCK REPORT: Day-trading ******

                Query Conditions:
                -----------------
                EPS latest quarter > 1.2 * EPS preceding year quarter AND
                EPS latest quarter > 0 AND
                YOY Quarterly sales growth > 25 AND
                EPS last year > EPS preceding year AND
                EPS > EPS last year AND
                Profit growth 3Years > 25 AND
                Return on equity > 17 AND
                Down from 52w high < 18 AND
                Market Capitalization > 5 AND
                ( Current assets / Current liabilities) > 2

                Retrieved 7 stock entries.

                Data Table:
                -----------
                +---------------+---------+------------------+---------------------------+------------------------------+-------------+-------------------------+-------------------------+-----

INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 05:09:44] "POST /stockRecommandation HTTP/1.1" 200 -




# Agent: Sentiment and Recommendation Analyst
## Final Answer: 
Based on the sentiment analysis and financial metrics, the top recommended stock is Espire Hospi (ESPIRE.BO) due to positive sentiment and leadership changes. Other stocks lack sentiment data but show potential based on financial metrics. Risk assessment suggests high risk, aligning with the user's risk tolerance. Regular monitoring is advised.
```


Received content: {'income': '100000', 'expenses': '12000', 'savings': '1000', 'investment_amount': '40000', 'risk_tolerance': 'Middle', 'strategy': 'Scalping'}
# Agent: Financial Analyst
## Task: Analyze the user's financial condition metrics and determine the budget and risk level for investment.
    User financial metrics:
    - Income: 100000
    - Expenses: 12000
    - Savings: 1000
    - Investment Amount: 40000
    - Risk Tolerance: Middle

    Provide a concise analysis of the user's financial health and recommend a budget and risk level for investment.
    Include 2

INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 05:48:20] "POST /stockRecommandation HTTP/1.1" 200 -




# Agent: Sentiment and Recommendation Analyst
## Final Answer: 
Top 5 Stock Recommendations Based on Sentiment and Financial Metrics

1. **Denta Water (DENTA.NS)**
   - **Financial Metrics**: P/E Ratio 56.36, Market Cap 952.79, No Dividend Yield
   - **Sentiment Analysis**: Positive sentiment with articles highlighting strong financials and growth.
   - **Recommendation**: Suitable for scalping due to high trading volume and positive sentiment. Moderate risk due to high P/E ratio.

2. **Zen Technologies (ZENTEC.NS)**
   - **Financial Metrics**: P/E Ratio 1087.80, Market Cap 9821.78, Dividend Yield 33.01
   - **Sentiment Analysis**: Positive sentiment due to recent product launches.
   - **Recommendation**: High growth potential with positive sentiment. Moderate to high risk due to high P/E ratio.

3. **APL Apollo Tubes (APLAPOLLO.NS)**
   - **Financial Metrics**: P/E Ratio 1489.20, Market Cap 41328.96, Dividend Yield 22.05
   - **Sentiment Analysis**: Positive sentiment with expandin